In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet18, resnet50
from torchvision.datasets import VOCDetection, Caltech256
from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm
import xml.etree.ElementTree as ET
import numpy as np
from transformers import get_linear_schedule_with_warmup
from torcheval.metrics.functional import multiclass_f1_score, multiclass_accuracy

In [2]:
torch.manual_seed(12345)

In [3]:
def get_center_updated(target):
    image_width= int(target['annotation']['size']['width'])
    image_height= int(target['annotation']['size']['height'])
    max_area_class = ''
    max_area = 0
    max_area_coordinates = None


# Find the object with the largest bounding box area
    for i in range(len(target['annotation']['object'])):
        coord = target['annotation']['object'][i]['bndbox']
        area = (int(coord['xmax']) - int(coord['xmin'])) * (int(coord['ymax']) - int(coord['ymin']))

        if area > max_area:
            max_area = area
            max_area_class = target['annotation']['object'][i]['name']
            max_area_coordinates = coord

    if max_area_coordinates is not None:
        # Calculate the size of the square (maximum of width and height of the bounding box)
        square_size = max(int(max_area_coordinates['xmax']) - int(max_area_coordinates['xmin']),
                          int(max_area_coordinates['ymax']) - int(max_area_coordinates['ymin']))

        # Calculate the coordinates for the square centered on the bounding box
        center_x = (int(max_area_coordinates['xmin']) + int(max_area_coordinates['xmax'])) // 2
        center_y = (int(max_area_coordinates['ymin']) + int(max_area_coordinates['ymax'])) // 2
        half_size = square_size // 2
        new_xmin = max(0, center_x - half_size)
        new_xmax = min(image_width, center_x + half_size)
        new_ymin = max(0, center_y - half_size)
        new_ymax = min(image_height, center_y + half_size)

        # Return the coordinates of the largest square including the bounding box
        square_coordinates = {
            'xmin': str(new_xmin),
            'ymin': str(new_ymin),
            'xmax': str(new_xmax),
            'ymax': str(new_ymax)
        }
        return square_coordinates, max_area_class

    else:
        return None

In [4]:
class ModifiedCaltech256Dataset(Caltech256):
    def __init__(self, path='./data', download_flag=False, validation_flag=False):
        super(ModifiedCaltech256Dataset, self).__init__(path, None, None, download_flag)
        self.classes = []
        self.labels = []
        # self.means = torch.zeros((3, 1), dtype=torch.float32)
        # self.vars = torch.zeros((3, 1), dtype=torch.float32)
        self.means = torch.from_numpy(np.array([[0.5520],
                                    [0.5336],
                                    [0.5050]], dtype=np.float32))
        self.means = torch.from_numpy(np.array([[0.1973],
                                    [0.1930],
                                    [0.2098]], dtype=np.float32))
        if validation_flag:
            self.transform_mod = transforms.Compose([
                transforms.Resize((224, 224)),
                # transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0), ratio=(0.8, 1.2)),
                # transforms.RandomHorizontalFlip(),
                # transforms.RandomRotation((-45., +45.)),
                transforms.ToTensor()
            ])
        else:
            self.transform_mod = transforms.Compose([
                # transforms.Resize((256, 256)),
                transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0), ratio=(0.8, 1.2)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation((0., 45.)),
                transforms.ToTensor()
            ])
    
    def __getitem__(self, index):
        img, target = super(ModifiedCaltech256Dataset, self).__getitem__(index)
        img_tensor = self.transform_mod(img)
        if img_tensor.dim()==2:
            img_tensor = img_tensor[None, ...].expand(3, -1, -1)
        elif img_tensor.shape[0]==1:
            img_tensor = img_tensor.expand(3, -1, -1)
        self.labels.append(target)
        img_tensor = transforms.Normalize(mean=[0.5520, 0.5336, 0.5050], std=[0.1973, 0.1930, 0.2098])(img_tensor)
        # self.means += torch.mean(img_tensor.reshape(3, -1), dim=1, keepdim=True)/super(ModifiedCaltech256Dataset, self).__len__()
        # self.vars += torch.pow(torch.mean(img_tensor.reshape(3, -1), dim=1, keepdim=True)-self.means, 2)/super(ModifiedCaltech256Dataset, self).__len__()
        return img_tensor, target

In [5]:
class ModifiedVOCDataset(VOCDetection):
    def __getitem__(self, index):
        img, target = super(ModifiedVOCDataset, self).__getitem__(index)

        # Use your function to get the bounding box
        square_coordinates, label = get_center_updated(target)

        # If we get valid coordinates, crop the image
        if square_coordinates:
            img = img.crop((int(square_coordinates['xmin']), int(square_coordinates['ymin']),
                            int(square_coordinates['xmax']), int(square_coordinates['ymax'])))

        # Convert label to index (based on VOC class names)
        classes = ('aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
                   'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
                   'dog', 'horse', 'motorbike', 'person', 'pottedplant',
                   'sheep', 'sofa', 'train', 'tvmonitor')
        label_idx = classes.index(label)

        # Apply transforms
        transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0), ratio=(0.8, 1.2)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation((-45., +45.)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        return transform(img), label_idx

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# Download and prepare the dataset
dataset = ModifiedCaltech256Dataset('./data', True)
# dataset = ModifiedVOCDataset(root='./data', year='2012', image_set='trainval', download=True)
# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=False, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8)

Files already downloaded and verified


In [8]:
# Model setup
model = resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 257)  # 20 classes in Pascal VOC
model.to(device)
model.train()

for param in model.parameters():
    if param.requires_grad and param.dim()>1:
        nn.init.xavier_normal_(param)

/home/ragraw06/UBCO/rishabh/AI_conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ragraw06/UBCO/rishabh/AI_conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
def save_model(model, optimizer, lr_scheduler, epoch, model_path):
    try:
        torch.save({'epoch':epoch,
                    # 'config':vars(config),
                    'state_dict':model.state_dict(),
                    'optimizer':optimizer.state_dict(),
                    'scheduler':lr_scheduler.state_dict()}, model_path)
    except Exception as err:
        print(f"Unexpected {err=}, {type(err)=}")
        return False
    return True

def load_model(model, optimizer, lr_scheduler, model_path):
    check_point = torch.load(model_path)
    epoch = check_point['epoch']
    # print(check_point['config'])
    config = Namespace(**check_point['config'])
    model.load_state_dict(check_point['state_dict'])
    optimizer.load_state_dict(check_point['optimizer'])
    lr_scheduler.load_state_dict(check_point['scheduler'])
    return model, optimizer, lr_scheduler, epoch

In [10]:
all_labels = []
for i, (images, labels) in enumerate(tqdm(train_loader)):
    all_labels.append(labels)
all_labels = torch.cat(all_labels, dim=0).numpy()
print(all_labels.shape)
uniques, counts = np.unique(all_labels, return_counts=True)

100%|█████████████████████████████████████████████████████████████████████████████████| 192/192 [00:15<00:00, 12.29it/s]

(24485,)


In [11]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(all_labels, bins=np.arange(len(uniques)+1))
# plt.show()

In [12]:
weights = np.zeros((len(uniques)), dtype=np.float32)
uniques, counts = np.unique(all_labels, return_counts=True)
weights[uniques] += counts
weights = np.sum(counts)/(len(uniques)*weights)
weights = torch.from_numpy(weights).to(device)
print(weights)

tensor([1.2060, 1.1762, 0.7939, 0.9924, 0.8583, 1.4220, 1.0826, 0.5068, 1.1479,
        1.3610, 0.4272, 0.5293, 1.1078, 1.3419, 0.9722, 1.2703, 1.1762, 1.1209,
        0.9722, 1.3610, 0.8822, 1.3419, 1.1078, 1.0244, 1.0135, 1.1342, 1.2214,
        1.0244, 1.2214, 1.1479, 1.2536, 1.1078, 1.1479, 1.2214, 1.4886, 1.1479,
        0.9822, 1.1479, 1.4657, 0.8741, 1.5367, 1.2536, 0.9722, 0.9822, 1.3419,
        0.9433, 1.3232, 1.2214, 0.9822, 1.1479, 1.0826, 1.3808, 1.4435, 1.0356,
        1.2373, 1.1909, 1.1078, 1.2536, 1.3419, 1.4011, 1.2373, 1.3610, 1.0135,
        0.8822, 1.1619, 1.3610, 1.4011, 1.0469, 1.0951, 1.3419, 1.2214, 1.0586,
        1.2703, 1.0586, 1.5123, 1.3808, 1.4220, 1.4220, 1.2060, 1.0244, 1.1619,
        1.4657, 1.3808, 1.4220, 1.0826, 1.5367, 1.2060, 1.1619, 1.0951, 0.5571,
        1.2214, 0.5992, 1.0826, 1.2060, 1.2875, 0.4054, 1.3419, 1.1619, 1.4011,
        1.2536, 0.8661, 1.4435, 0.9924, 1.2373, 0.4390, 1.3610, 1.5879, 1.3808,
        0.7939, 1.3232, 1.4220, 1.4886, 

In [13]:
num_epochs = 100
warmup_frac = 0.2
total_iterations = num_epochs * len(train_loader)
print(total_iterations)

19200


In [14]:
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=1e-5)
# lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5, threshold=1e-3,
#                                                            min_lr=1e-8, eps=1e-08, verbose=True)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, 
                                               int(warmup_frac*total_iterations), 
                                               total_iterations
                                               # total_iterations-int(warmup_frac*total_iterations)
                                              )

# Training and Validation Loop
val_acc = 0
for epoch in range(num_epochs):
    model.train()
    correct_train = 0
    total_train = 0
    train_loss = 0
    for i, (images, labels) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        train_loss += loss.cpu().item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        predicted = torch.argmax(torch.softmax(outputs.data, dim=1), dim=1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        lr_scheduler.step()
        
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0
    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(val_loader)):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            predicted = torch.argmax(torch.softmax(outputs.data, dim=1), dim=1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            val_loss += loss.cpu().item()

    val_loss /= len(val_loader)
    train_loss /= len(train_loader)
    # lr_scheduler.step(val_loss)
    if val_acc<=(correct_val / total_val):
        val_acc = correct_val / total_val
        print('Saving Model {} ...'.format('./sc_resnet50_voc2012.pth'))
        save_model(model, optimizer, lr_scheduler, epoch, './sc_resnet50_voc2012.pth')
    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Train Accuracy: {(correct_train / total_train) * 100:.2f}%, "
          f"Validation Accuracy: {(correct_val / total_val) * 100:.2f}% "
          f"\nLoss Train: {train_loss} | Val: {val_loss}")

print("Training complete.")

100%|███████████████████████████████████████████████████████████████████████████████████| 48/48 [00:05<00:00,  8.18it/s]


Saving Model ./sc_resnet50_voc2012.pth ...
Epoch [1/100], Train Accuracy: 0.65%, Validation Accuracy: 1.11% 
Loss Train: 5.868919104337692 | Val: 5.458239843448003


100%|███████████████████████████████████████████████████████████████████████████████████| 48/48 [00:05<00:00,  8.04it/s]


Saving Model ./sc_resnet50_voc2012.pth ...
Epoch [2/100], Train Accuracy: 3.09%, Validation Accuracy: 4.62% 
Loss Train: 5.270129981140296 | Val: 5.298472175995509


100%|███████████████████████████████████████████████████████████████████████████████████| 48/48 [00:05<00:00,  8.18it/s]


...

Epoch [98/100], Train Accuracy: 100.00%, Validation Accuracy: 63.29% 
Loss Train: 0.0008715434629872713 | Val: 2.9786045302947364


100%|███████████████████████████████████████████████████████████████████████████████████| 48/48 [00:06<00:00,  7.61it/s]


Epoch [99/100], Train Accuracy: 99.98%, Validation Accuracy: 63.21% 
Loss Train: 0.0009860162598063955 | Val: 2.9773538758357367


100%|███████████████████████████████████████████████████████████████████████████████████| 48/48 [00:06<00:00,  7.65it/s]

Epoch [100/100], Train Accuracy: 100.00%, Validation Accuracy: 62.96% 
Loss Train: 0.0006446068786090109 | Val: 2.958790351947149
Training complete.


In [15]:
print('Best Validation Accuracy: {:.2f}%'.format(val_acc*100))

Best Validation Accuracy: 63.42%
